In [1]:
import itertools
import math
import pickle
import time

from simulation import simulate
from algs import *
from payoff import *
from plotting import *
from util import *

import toyplot as tp

In [2]:
steps = 2000
narms = 2
nsims = 4

strategies = [ucb]
nagents = [2000, 500, 100, 50]
lifespans = [5, 20, 50, 200]

# Negative externalities

$$ Q(\theta, f) = \frac{\theta}{1+Lf}$$

In [3]:
def neg_ext(Ls, strategies, lifespans, nagents, steps=2000):
    t_start = time.time()
    
    size_cross = len(Ls) * len(strategies) * len(lifespans) * len(nagents)
    print("Starting {} simulations".format(size_cross))

    for itr, (L, strategy, lifespan, nagent) in enumerate(itertools.product(Ls, strategies, lifespans, nagents)):
        config = {"k": narms, 
                  "types": [(3, 1), (1, 3)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "payoff": create_neg_externatilities(L)}

        fname = "neg_ext_L{}_S{}_A{}_K{}_LS{}".format(int(10*L),
                                                      config["strategy"].__name__, 
                                                      config["nagents"], 
                                                      config["k"], 
                                                      lifespan)

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [4]:
# Figure 1
neg_ext([0.2, 1, 3, 7, 20], [ucb], [50], [2000], steps)

Starting 5 simulations
	  40.0% 	 372.42s 	 558.63s
	  60.0% 	 555.21s 	 370.14s
	  80.0% 	 734.05s 	 183.51s
	 100.0% 	 920.11s 	  0.00s
Total time: 920.11 seconds


In [5]:
# Figure 2
neg_ext([20], [ucb], [200], nagents, steps)

Starting 4 simulations
	  50.0% 	 259.10s 	 259.10s
	  75.0% 	 268.23s 	 89.41s
	 100.0% 	 273.32s 	  0.00s
Total time: 273.32 seconds


# Positive externalities

In [6]:
def pos_ext(strategies, nagents, lifespans, steps=2000):
    size_cross = len(strategies) * len(nagents) * len(lifespans)
    t_start = time.time()
    print("Starting {} simulations".format(size_cross))

    for itr, (strategy, nagent, lifespan) in enumerate(itertools.product(strategies, nagents, lifespans)):
        config = {"k": narms, 
                  "types": [(1, 1), (1, 1)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "payoff": pos_externalities}

        fname = "pos_ext_S{}_A{}_K{}_LS{}".format(config["strategy"].__name__, 
                                                    config["nagents"], 
                                                    config["k"], 
                                                    lifespan)

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [7]:
# Figure 3 and 4
pos_ext([ucb], nagents, [10, 200], steps)

Starting 8 simulations
	  25.0% 	 349.44s 	 1048.31s
	  37.5% 	 390.02s 	 650.03s
	  50.0% 	 431.96s 	 431.96s
	  62.5% 	 440.80s 	 264.48s
	  75.0% 	 450.12s 	 150.04s
	  87.5% 	 454.30s 	 64.90s
	 100.0% 	 458.75s 	  0.00s
Total time: 458.75 seconds


# Coordination game

In [9]:
def coord_game(gammas, strategies, nagents, lifespans):
    size_cross = len(gammas) * len(strategies) * len(nagents) * len(lifespans)
    t_start = time.time()
    print("Starting {} simulations".format(size_cross))

    for itr, (gamma, strategy, nagent, lifespan) in enumerate(itertools.product(gammas, strategies, nagents, lifespans)):
        config = {"k": narms, 
                  "types": [(1, 1), (1, 1)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "payoff": create_coord_game(gamma)}

        fname = "coordgame_G{}_S{}_A{}_K{}_LS{}".format(int(gamma*100),
                                                      config["strategy"].__name__, 
                                                      config["nagents"], 
                                                      config["k"], 
                                                      lifespan)

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [10]:
# Figure 5
coord_game([0.5], [ucb], [2000], lifespans)

Starting 4 simulations
	  50.0% 	 338.38s 	 338.38s
	  75.0% 	 510.66s 	 170.22s
	 100.0% 	 686.23s 	  0.00s
Total time: 686.23 seconds
